# Example limit setting code

In [ ]:
from __future__ import print_function
%matplotlib inline

Dependencies

In [ ]:
import functools
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import brenth
import pprint
from scipy.special import erf
import cmax

The following line loads from disk (if available) precomputed constants for running the optimum interval calculation.

In [ ]:
short_run = False
if short_run:
    run = '7days'
    livetime = 7.2
else:
    run = 'run10'
    livetime = 223.46

qy_cutoff = 1.0

shasum used to check you have correct file:

d80d5d2856e3477c74102e9da0b5c7ef46016f5d  trim_cut_s2only_dm_7days.root

66377496c6627ca6b1351b8a04ff356cc30940f0  trim_cut_s2only_dm_run10.root

Available at xecluster:~tunnell/trim_files/

In [ ]:


import ROOT
from root_numpy import tree2rec

f = ROOT.TFile('trim_files/trim_cut_s2only_dm_%s.root' % run)
rec = tree2rec(f.Get('T2'))
xe100_unblinded = np.array([x[0] for x in rec['S2sTot']])

In [ ]:
len(xe100_unblinded)

In [ ]:
size = 3.1693
plt.figure(figsize=(size, 0.75*size))
plt.hist(xe100_unblinded, histtype='step', bins=50)
plt.xlabel('Measured energy [pe]')
plt.ylabel('#')
plt.savefig('spectrum.png')
plt.savefig('spectrum.eps', bbox_inches='tight')
plt.savefig('spectrum.pdf', bbox_inches='tight')

In [ ]:
cmax.load_table_from_disk()
n = 100

# Inputs to the analysis

### Response matrix

This is computed elsewhere (see the respective IPython notebook) and provides a response matrix.  This matrix can be multipled against a histogram in true recoil energy to yield the observed spectra.

$\begin{pmatrix}E^T_0\\E^T_1\\...\\E^T_n\end{pmatrix} \begin{pmatrix}response matrix\end{pmatrix} = \begin{pmatrix}E^M_0\\E^M_2\\...\\E^M_n\end{pmatrix}$

where $E^T$ and $E^M$ are true and measured energies, respectively.

In [ ]:
# Load the dictionary back from the pickle file.
f = open("response_matrix_%0.1f.p" % qy_cutoff, "rb")
true_energy_bins = pickle.load(f)
measured_energy_bins = pickle.load(f)
response_matrix = pickle.load(f)
f.close()

measured_energy_bins_scale = (measured_energy_bins[-1] - measured_energy_bins[0])/measured_energy_bins.size
true_energy_bins_scale = (true_energy_bins[-1] - true_energy_bins[0])/true_energy_bins.size


### Fundumental constants

In [ ]:
xenon_liquid_density = 2.91 / 1000 # kg / cm^2
A = 131.293 # amu
v = 230 * 100000 # 230 km/s, in cm/s
speed_of_light = 2.99792458e8 # m/s, light

rho = 0.3 # in GeV cm^-3
avogadro_number = 6.0221415 * 10**26 # 1/kg

mnucl = 0.931494    # Mass of a nucleon, in GeV/c^2
mprot = 0.938272046 # mass of proton, GeV/c^2

A = 131.293 # amu
vsun = 232.0 # Solar velocity in km/s
vinf = 220.0 # Asymptotic velocity of local system in km/s
vesc = 544.0 # Galactic escape velocity In km/s 

### Parameters of run

In [ ]:
print(xe100_unblinded)
print('Number of events', len(xe100_unblinded))

#Assuming the TPC is 30 cm high, our fiducial volume is 49.367 kg.  We have one inch PMTs, so assuming that signals in the top PMTs give our position, then the position uncertainty is $1 \text{ inch}/\sqrt{12} = 7 \text{ mm}$, where the $\sqrt{12}$ is the standard deviation of a uniform distribution (See, e.g., [[http://mathforum.org/library/drmath/view/52066.html|here]]).  

#Normally, our fiducial is 33.9292 L of xenon.  If we increase or decrease our cut value of $r$ to understand how uncertainties in $r$ affects our fiducial volume, we get 37.562 L and 30.481 L, respectively.  This translates into an 11% uncertainty on our fiducial volume, which is roughly our statistical uncertainty on 100 photoelectron signals.
radius_cut = np.sqrt(18000) / 10 # cm
target_mass = ((np.pi * radius_cut ** 2) * 30) * xenon_liquid_density # mm^2 -> cm^2 * 30 cm * density
print('Target mass [kg]:', target_mass)

### Plotting setting

In [ ]:
plot_ranges = {'mass_min' : 4, # GeV
              'mass_max' : 20, # GeV
              'mass_steps' : 20,
              'xsec_min' : 10**-43, # cm^-2
              'xsec_max' : 10**-39,} 

# 'Experts' only below

### Helpers

The following few functions are just handy to define, but don't affect the physics.

In [ ]:
def center(x):
    """returns bin centers"""
    return 0.5*(x[1:]+x[:-1])

In [ ]:
# Test values for quick checks
test_mass = 10.0 # GeV
test_xsec = 10**-41 # cm^2

In [ ]:
def ErfMinusErf(x, y):
    retval = np.sqrt(np.pi)/2
    retval *= (erf(y)-erf(x))
    return retval

### Interaction spectrum and normalization

All of this code is essentially stolen from Andy's LimitSimulator3 code.  My function has been tested versus his function and give identical answers, if you assume the form factor is 1.  This is true at these low energies (>0.9).

From Donato et al ([arXiv:9803295](http://arxiv.org/pdf/hep-ph/9803295v1.pdf)):

By means of the previous  definitions, the  differential rate can be written in
the  form

$$  \frac   {dR}{dE_R}=   N_{T}
\frac{\rho_{\chi}}{m_{\chi}}\frac{m_N \sigma_0}  {2   m_{\rm   red}^2}
F^{2}(q^2)\, {\cal  I}(v_{\rm min},v_\odot,v_{\rm esc})
$$

where the  function ${\cal  I}(v_{\rm min},v_\odot,v_{\rm esc})$ contains all
the details of the integration of the velocity DF $f(v)$
in the Earth's frame.



In [ ]:
def GetI(erec, Mchi, mred, Mnucleus):
    """
    erec - recoil energy (keV)
    Mchi - DM mass (GeV)
    mred - reduced mass (GeV)
    """    
    neta = vsun / vinf # Unitless
    z = vesc / vinf # Unitless
    
    # sqrt(keV * GeV * (km/s) * (km/s) / GeV**2 / (km/s))
    # = sqrt(keV * (km/s) / GeV) -> so 1e-6
    xmin =  np.sqrt(erec * Mnucleus * speed_of_light * speed_of_light * 1e-12/ (2*mred*mred)) / vinf 
    
    norm = 1.0/(erf(z)-(2/np.sqrt(np.pi))*z*np.exp(-z*z))

    retval = (norm/neta)/(np.sqrt(np.pi) * vinf)
    
    if xmin < (z-neta):
     retval *= ErfMinusErf(xmin-neta,xmin+neta)-2*neta*np.exp(-z*z);
    if ((z-neta)<=xmin and xmin < (z+neta)):
      retval *= ErfMinusErf(xmin-neta,z)-np.exp(-z*z)*(z+neta-xmin)
    if (xmin>=(z+neta)):
        retval = 0

    return retval

Mnucleon = 0.931494    # Mass of a nucleon, in Gev/c^2

def dRdE(erec, Mchi, sigma):
    """
    erec - (keV)
    Mchi - Mass of WIMP (GeV)
    sigma - cross section nucleon (cm^-2)
    """
    # Helpers
    Mnucleus = A * Mnucleon # GeV
    
    Nt = avogadro_number / A # #, Number of target nuclei per unit of max

    #Returns the per nucleon scale factor for the know masses
    #Again following Lewin and Smith 1996
    sigma /= ((1 + Mchi/Mnucleus)/(1 + Mchi/mprot))**2
    sigma *= A**2
    
    mred = (Mnucleus * Mchi) / (Mnucleus + Mchi)  # Reduced mass
    
    F2 = 1  # Form factor squared, true only for low mass analysis!
    
    I = 1e-5 * GetI(erec, Mchi, mred, Mnucleus) # Integrate velocity dist, then km -> cm
    
    # Notice the Mn/med**2 in the returned formula? Convert to 1/keV
    scale = 1e-6  # 1e-6 takes 1/GeV
    scale *= speed_of_light**2 * 1e4 # c^2 in cm/s
    
    scale *= 60 * 60 * 24 # formula in paper yields days, want seconds
        
    return Nt * (rho/Mchi) * ((Mnucleus * sigma)/(2 * mred**2)) * F2 * I * scale
    

assert (dRdE(5.0, test_mass, 1e-36) - 9434.53) < 1 # Test against Andrew's code

In [ ]:
test_mass = 5

plt.plot(true_energy_bins, [1 * 1 * dRdE(y, test_mass, test_xsec) for y in true_energy_bins], label='other')
plt.title('Recoil spectra for %d GeV, %g cm^2' % (test_mass, test_xsec))
plt.xlabel('True energy [keV]')
plt.ylim(1e-3, 1e2)
plt.ylabel('dN/dE [1/keV/day/kg]')
plt.yscale('log')
plt.legend()
plt.show()
#plt.savefig('spectrum.eps')
#plt.savefig('spectrum.png')


We can now use our response matrix to determine the reconstructed spectra.


In [ ]:
def get_recon_spectra(Mchi, sigma):
    """Need to multiply against norm"""
    # True dist
    distribution_binned = [dRdE(energy, Mchi, sigma) for energy in center(true_energy_bins)]
    distribution_binned = np.array(distribution_binned)
    distribution_binned *= true_energy_bins_scale
    
    # Convert to matrix so can matrix multiply
    true_vector = np.mat(distribution_binned)
    recon_vector = true_vector * response_matrix
    recon_vector = recon_vector.getA()[0]
    
    return recon_vector


plt.plot(center(measured_energy_bins),
         get_recon_spectra(test_mass, test_xsec))
plt.xlabel('energy [pe]')
plt.ylabel('events per %d pe' % measured_energy_bins_scale)

test_spectra = get_recon_spectra(test_mass, test_xsec)

print('sum_after', sum(test_spectra))


The CDF of the measured recoil spectra is needed for the limit setting code.

In [ ]:
def ReconWimpCDF(list_of_energies,
                 xbins,
                 Mchi):
    
    x = center(xbins)
    y = np.cumsum(get_recon_spectra(Mchi, 1e-36))
    y /= y[-1]
    
    f = interp1d(center(measured_energy_bins),
                 y,
                 kind='cubic',
                bounds_error=False,
                  fill_value=0)
        
    return f(list_of_energies)


ReconWimpCDF(xe100_unblinded, center(measured_energy_bins), 10)

Max gap eq (not used) $C_0(x, \mu) = \sum_{k=0}^{m} \frac{(kx - \mu)^k e^{-kx}}{k!} (1 + \frac{k}{\mu - kx}) $

In [ ]:
mc_test_vals = np.array([168.753618,115.627839,264.770982,123.617452,517.887246,244.728130,138.523418,101.538961,130.015155,119.991030,295.611937,164.026573,138.648365,151.799804,343.600605,118.806085,153.343472,273.649671,155.601161,103.716820,206.913062,362.686804,94.494276,196.579429,89.072226,140.539444,282.166358,231.011781,159.959681,182.687679,231.234782,142.422258,126.581473,134.855876,175.179386,138.506776,139.365890,103.367954,144.970514,105.035875,246.662424,137.983437,184.380137,219.589717,146.567977,50.458623,232.625746,288.887184,268.107974,200.472974,151.678259,311.605239,139.888106,116.270688,124.973920,143.058264,443.803889,110.264946,395.976192,205.320753,130.478291,193.603740,253.107474,163.382504,113.357371,274.718855,97.268250,197.657366,510.470246,169.722057,235.607320,167.766701,142.263224,233.123329,92.945875,519.793448,364.691687,188.154327,])

In [ ]:
#print('Precision test:')
#f3 = lambda x: ReconWimpCDF(x, measured_energy_bins, test_mass)

#results = {}
#for test_n in [100, 500, 1000, ]:
#    results[test_n] = cmax.optItvUpperLimit(xe100_unblinded, 0.9, spectrumCDF=f3, n=test_n)
    
#pprint.pprint(results)


In [ ]:
#f = lambda x: ReconWimpCDF(x, measured_energy_bins, mass)

#for mu in [3000, 4000, 5000, 6000]:
#    print(mu, cmax.optItvUpperLimit(xe100_unblinded, 0.9, spectrumCDF=f,
#                                   n=n,
#                                    mu=mu))

In [ ]:
limit_mass = np.logspace(np.log10(plot_ranges['mass_min']),
                         np.log10(plot_ranges['mass_max']),
                         plot_ranges['mass_steps'])
limit_sec = []

for mass in limit_mass:

    f = lambda x: ReconWimpCDF(x, measured_energy_bins, mass)

    answer = cmax.optItvUpperLimit(xe100_unblinded, 0.9, spectrumCDF=f, n=n)
    
    print("Number of events U.L.", answer)
    
    def f2(xsec, mass, answer):
        xsec = 10**xsec
        nevts = sum(get_recon_spectra(mass, xsec)) 
        nevts *= livetime * target_mass
        print('\ttest: xsec', xsec, 'mass:', mass, 'nevts:', nevts, 'nevts', nevts - answer)
        return nevts - answer

    try:
        xsec = brenth(f2, np.log10(plot_ranges['xsec_min']),
                      np.log10(plot_ranges['xsec_max']),
                      args=(mass, answer))    
    except:
        print('Mass:', mass, 'Cross section:', 'outside plot range')
        limit_sec.append(1)
        continue
        
    print('Mass:', mass, 'Cross section:', xsec)

    limit_sec.append(10**xsec)
    cmax.write_table_to_disk()

    

In [ ]:
cmax.write_table_to_disk()

In [ ]:
pprint.pprint(list(zip(limit_mass, limit_sec)))

In [ ]:
print('limit_mass = ', list(limit_mass))
print('limit_sec = ', limit_sec)

## Quick XENON10 cross check

In the XENON10 S2-only analysis, it states that the limit at $M_\chi = 7 \text{ GeV}$ is $\sigma = 10^{-42} \text{ cm}^{2}$.  I try to reproduce this.

* XENON10 efficiency: 0.8
* XENON10 number of events: 23
* XENON10 exposure: 15 kg-days

I expect 9 events, whereas they measure 23.  I think this is due to the fact that they used the $p$ method, therefore their total number of events for the limit setting is between 0 and 23.  I guessing their used the $p$ method to set (in an unbiased way) bin boundaries that only accepted 9 events.


In [ ]:
def check_xenon10():
    xenon10_measurement = 23

    mass = 7
    xsec = 7e-42
    nevts = sum(get_recon_spectra(mass, xsec)) 
    nevts *= 15 # exposure in kg-days
    nevts *= 1.6 # 1.6 from 0.8/0.5 efficiency ratio
    
    print('Mass:', mass, 'GeV')
    print('Cross section:', xsec, 'cm^2')
    print('\tXENON10 measurement:', xenon10_measurement)
    print('\tMy predicted number of events:', nevts)

check_xenon10()

In [ ]:
cmax.write_table_to_disk()